In [ ]:
import random

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_percentage_error, r2_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
topics_over_time = pd.read_csv(
    '/content/drive/MyDrive/Colab Notebooks/DTM_collab_dataset_small_07_22_<220000_manual_sentence-transformers_default.csv')


In [ ]:
# Specifying how many values to predict
time_step = 1
topic_number = 10


In [ ]:
topics_over_time.shape


In [ ]:
topics_over_time = topics_over_time.sort_values(by=['Topic', 'Timestamp'])


In [ ]:
topics_over_time['Frequency_Next_Year'] = topics_over_time.groupby('Topic')['Frequency'].shift(-1)
topics_over_time['Lag-1'] = topics_over_time.groupby('Topic')['Frequency'].shift(1)
topics_over_time['Diff-1'] = topics_over_time.groupby('Topic')['Frequency'].diff(1)
topics_over_time['Rolling-4'] = topics_over_time.groupby(
    'Topic')['Frequency'].rolling(4).mean().reset_index(level=0, drop=True)


In [ ]:
topics_over_time[topics_over_time['Topic']==topic_number]

In [ ]:
topics_index = set(topics_over_time['Topic'].unique())
training_topics_index = random.sample(topics_index, 90)
training = topics_over_time[topics_over_time['Topic'].isin(training_topics_index)]
testing = topics_over_time[~(topics_over_time['Topic'].isin(training_topics_index))]
training = training.dropna()


In [ ]:
training.head()

In [ ]:
testing.head()

In [ ]:
training[training['Topic'] == topic_number]


In [ ]:
features = ['Frequency', 'Lag-1', 'Diff-1', 'Rolling-4']


In [ ]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(training[features])
ytr = training['Frequency_Next_Year']

mdl = RandomForestRegressor(n_estimators=600, random_state=0, n_jobs=16)
mdl.fit(Xtr, ytr)


In [ ]:
testing = testing.dropna(subset=['Frequency_Next_Year'])
Xval = imputer.transform(testing[features])
yval = testing['Frequency_Next_Year']

p = mdl.predict(Xval)


In [ ]:
t_testing = testing.dropna()
mean_absolute_percentage_error(t_testing['Frequency_Next_Year'], t_testing['Frequency'])


In [ ]:
print(mean_absolute_percentage_error(yval, p))
print(yval[:10])
print(p[:10])


In [ ]:
r2_score(yval, p)
